# Project - Airline AI Assistant

We'll now bring together what we've learned to make an AI Customer Support assistant for an Airline

In [1]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import sqlite3

In [45]:
load_dotenv(override=True)
openai_api_key=os.getenv("OPENAI_API_KEY")
if(openai_api_key):
    print(f"open ai API Key starts with {openai_api_key[:8]}")
else:
    print(f"API Key does not exist")

MODEL="gpt-4.1-mini"
openai = OpenAI()
DB = "prices.db"


open ai API Key starts with sk-proj-


In [46]:
system_message = """
You are a helpful assistant for an Airline called FlightAI.
Give short, courteous answers, no more than 1 sentence.
Always be accurate. If you don't know the answer, say so.
"""

In [47]:
def get_ticket_price(city):
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('SELECT price FROM prices WHERE city = ?', (city.lower(),))
        result = cursor.fetchone()
        return f"Ticket price to {city} is {result[0]}" if result else "No price data available for this city"

In [40]:
get_ticket_price("hyderabad")

'Ticket price to hyderabad is $700'

# Apply tools for the get ticket price

In [64]:
price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}
tools = [{"type": "function", "function": price_function}]
tools

[{'type': 'function',
  'function': {'name': 'get_ticket_price',
   'description': 'Get the price of a return ticket to the destination city.',
   'parameters': {'type': 'object',
    'properties': {'destination_city': {'type': 'string',
      'description': 'The city that the customer wants to travel to'}},
    'required': ['destination_city'],
    'additionalProperties': False}}}]

In [49]:
def chat(message,history):
    hisotry=[{"role":h["role"],"content":h["content"]} for h in history]
    messages=[{"role":"system","content":system_message}] + history + [{"role":"user","content":message}]
    response=openai.chat.completions.create(model=MODEL,messages=messages)
    return response.choices[0].message.content 

gr.ChatInterface(fn=chat,type="messages").launch()

* Running on local URL:  http://127.0.0.1:7910
* To create a public link, set `share=True` in `launch()`.


In [66]:
def chat(message,history):
    history=[{"role":h["role"],"content":h["content"]} for h in history]
    messages=[{"role":"system","content":system_message}] + history + [{"role":"user","content":message}]
    response=openai.chat.completions.create(model=MODEL,messages=messages)
    while response.choices[0].finish_reason == "tool_calls":
        message = response.choices[0].message
        print(f"too call is been made for {message}")
        responses=handle_tool_calls(message)
        messages.append(message)
        messages.append(responses)
        print(f"appended response to the context window after the tool call {responses}")
        response=openai.chat.completions.create(model=MODEL,messages=messages,tools=tools)
    return response.choices[0].message.content

In [65]:
def handle_tool_calls(message):
    responses = []
    print(f"handle tool calls called")
    for tool_call in message.tool_calls:
        if tool_call.function.name == "get_ticket_price":
            arguments = json.loads(tool_call.function.arguments)
            city = arguments.get('destination_city')
            price_details = get_ticket_price(city)
            responses.append({
                "role" : "tool",
                 "content" : price_details,
                 "tool_call_id" : tool_call.id})
    return responses

In [67]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7915
* To create a public link, set `share=True` in `launch()`.
